In [1]:
import numpy as np
import pandas as pd
import json
import nltk
import string
import random
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model

In [2]:
content =  json.loads(open("content.json").read())

In [3]:
tags = []
inputs = []
responses = {}
for intent in content['intents']:
  responses[intent['tag']] = intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [4]:
data = pd.DataFrame({'inputs': inputs, 'tags': tags})
data

,inputs,tags
0,Hey,greetings
1,Hello,greetings
2,Hi,greetings
3,Greetings,greetings
4,Have a good day,goodbye
5,I am Leaving,goodbye
6,Goodbye,goodbye
7,Cya,goodbye
8,Bye,goodbye


In [5]:
data['inputs'] = data['inputs'].apply(lambda wrd: [ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
data

,inputs,tags
0,hey,greetings
1,hello,greetings
2,hi,greetings
3,greetings,greetings
4,have a good day,goodbye
5,i am leaving,goodbye
6,goodbye,goodbye
7,cya,goodbye
8,bye,goodbye


In [6]:
tokenizer = Tokenizer(num_words = 2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

In [7]:
X_train = pad_sequences(train)

In [8]:
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [9]:
input_shape = X_train.shape[1]
input_shape

4

In [10]:
vocabulary = len(tokenizer.word_index)
vocabulary

14

In [11]:
output_length = le.classes_.shape[0]
output_length

2

In [12]:
i = Input(shape = (input_shape,))
x = Embedding(vocabulary + 1, 10)(i)
x = LSTM(10, return_sequences = True)(x)
x = Flatten()(x)
x = Dense(output_length, activation = "softmax")(x)
model = Model(i, x)

In [13]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [14]:
train = model.fit(X_train, y_train, epochs = 200)

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 0.6928 - accuracy: 0.5556
Epoch 2/200
1/1 [==============================] - 0s 13ms/step - loss: 0.6920 - accuracy: 0.5556
Epoch 3/200
1/1 [==============================] - 0s 14ms/step - loss: 0.6913 - accuracy: 0.5556
Epoch 4/200
1/1 [==============================] - 0s 20ms/step - loss: 0.6905 - accuracy: 0.5556
Epoch 5/200
1/1 [==============================] - 0s 10ms/step - loss: 0.6897 - accuracy: 0.5556
Epoch 6/200
1/1 [==============================] - 0s 13ms/step - loss: 0.6889 - accuracy: 0.5556
Epoch 7/200
1/1 [==============================] - 0s 18ms/step - loss: 0.6881 - accuracy: 0.5556
Epoch 8/200
1/1 [==============================] - 0s 11ms/step - loss: 0.6873 - accuracy: 0.5556
Epoch 9/200
1/1 [==============================] - 0s 9ms/step - loss: 0.6864 - accuracy: 0.5556
Epoch 10/200
1/1 [==============================] - 0s 14ms/step - loss: 0.6856 - accuracy: 0.5556
Epoch 11/200
1/1 [====

In [17]:
while True:
  text_p = []
  prediction_input = input('You : ')

  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  text_p.append(prediction_input)

  prediction_input = tokenizer.texts_to_sequences(text_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input], input_shape)

  output = model.predict(prediction_input)
  output = output.argmax()

  response_tag = le.inverse_transform([output])[0]
  print("Alan :", random.choice(responses[response_tag]))
  if response_tag == 'goodbye':
    break

You : hi
Alan : Hello! I am Alan, Turing Club's virtual assistant. How can I can help you?
You : cya
Alan : Have a nice day!
